#### Atividade Fuzzy

Imagine que um fazendeiro planeja contratar um técnico para que ele automatize o sistema de irrigação de sua fazenda.

<br>
<img src='images/fazenda.jpg' width= 500px>

Este técnico planeja utilizar sistemas fuzzy para isso.

Para isso é importante levar em consideração 2 fatores para a construção desse modelo, a **umidade do solo** e a **temperatura do ar**.
Com isso, será possível usar um sistema fuzzy para saber a **duração da irrigação.**


#### Conjunto de regras

O técnico montou um conjunto de regras para fazer a inferência de seu sistema:

    Se a umidade do solo é baixa e a temperatura do ar é baixa, então a duração da irrigação deve ser média.

    Se a umidade do solo é baixa e a temperatura do ar é média, então a duração da irrigação deve ser longa.

    Se a umidade do solo é baixa e a temperatura do ar é alta, então a duração da irrigação deve ser longa.

    Se a umidade do solo é média e a temperatura do ar é baixa, então a duração da irrigação deve ser curta.

    Se a umidade do solo é média e a temperatura do ar é média, então a duração da irrigação deve ser média.

    Se a umidade do solo é média e a temperatura do ar é alta, então a duração da irrigação deve ser longa.

    Se a umidade do solo é alta e a temperatura do ar é baixa, então a duração da irrigação deve ser curta.

    Se a umidade do solo é alta e a temperatura do ar é média, então a duração da irrigação deve ser curta.

    Se a umidade do solo é alta e a temperatura do ar é alta, então a duração da irrigação deve ser média.

#### Funções de Pertinência

##### Antecedentes

<img src='images/umidade.png' width= 300px>
<img src='images/temperatura.png'width= 300px>

##### Consequentes

<img src='images/duracao.png' width= 300px>

##### Construa o sistema que o técnico imaginou

#### Criando antescedentes (entrada/input)

In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

umidade_solo = ctrl.Antecedent(np.arange(0, 101, 1), 'umidade_solo')
temperatura_ar = ctrl.Antecedent(np.arange(0, 41, 1), 'temperatura_ar')

umidade_solo['baixa'] = fuzz.trimf(umidade_solo.universe, [0, 0, 50])
umidade_solo['media'] = fuzz.trimf(umidade_solo.universe, [25, 50, 75])
umidade_solo['alta'] = fuzz.trimf(umidade_solo.universe, [50, 100, 100])

temperatura_ar['baixa'] = fuzz.trimf(temperatura_ar.universe, [0, 0, 20])
temperatura_ar['media'] = fuzz.trimf(temperatura_ar.universe, [10, 20, 30])
temperatura_ar['alta'] = fuzz.trimf(temperatura_ar.universe, [20, 40, 40])

#### Criando consequent (saída/output)

In [ ]:
duracao_irrigacao = ctrl.Consequent(np.arange(0, 61, 1), 'duracao_irrigacao')

duracao_irrigacao['curta'] = fuzz.trimf(duracao_irrigacao.universe, [0, 0, 30])
duracao_irrigacao['media'] = fuzz.trimf(duracao_irrigacao.universe, [15, 30, 45])
duracao_irrigacao['longa'] = fuzz.trimf(duracao_irrigacao.universe, [30, 60, 60])

#### Criando regras

In [ ]:
rule1 = ctrl.Rule(umidade_solo['baixa'] & temperatura_ar['baixa'], duracao_irrigacao['media'])
rule2 = ctrl.Rule(umidade_solo['baixa'] & temperatura_ar['media'], duracao_irrigacao['longa'])
rule3 = ctrl.Rule(umidade_solo['baixa'] & temperatura_ar['alta'], duracao_irrigacao['longa'])
rule4 = ctrl.Rule(umidade_solo['media'] & temperatura_ar['baixa'], duracao_irrigacao['curta'])
rule5 = ctrl.Rule(umidade_solo['media'] & temperatura_ar['media'], duracao_irrigacao['media'])
rule6 = ctrl.Rule(umidade_solo['media'] & temperatura_ar['alta'], duracao_irrigacao['longa'])
rule7 = ctrl.Rule(umidade_solo['alta'] & temperatura_ar['baixa'], duracao_irrigacao['curta'])
rule8 = ctrl.Rule(umidade_solo['alta'] & temperatura_ar['media'], duracao_irrigacao['curta'])
rule9 = ctrl.Rule(umidade_solo['alta'] & temperatura_ar['alta'], duracao_irrigacao['media'])

#### Criando sistema

In [ ]:
sistema_controle = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9])
simulacao = ctrl.ControlSystemSimulation(sistema_controle)

def calcular_duracao_irrigacao(umidade, temperatura):
    simulacao.input['umidade_solo'] = umidade
    simulacao.input['temperatura_ar'] = temperatura
    simulacao.compute()
    return simulacao.output['duracao_irrigacao']

umidade = 30 
temperatura = 25 
duracao = calcular_duracao_irrigacao(umidade, temperatura)
print(f'Duração da irrigação: {duracao:.2f} minutos')


# Testando o sistema de controle
valores_teste = [
    (20, 15),
    (60, 25),
    (80, 35)
]

for umidade, temperatura in valores_teste:
    duracao = calcular_duracao_irrigacao(umidade, temperatura)
    print(f'Umidade do solo: {umidade}%, Temperatura do ar: {temperatura}°C, Duração da irrigação: {duracao:.2f} minutos')
